In [187]:
import pandas as pd
import datetime
import numpy as np

In [202]:
root_dir='/Users/robinruhm/Documents/Research/Fire Hydrology/SlaterFireHydrology';
data_dir=root_dir+'/data';
processed_dir =data_dir+'/Processed';
raw_dir=data_dir+'/Raw';

In [203]:
#precip_data_fn=raw_dir+'/salmonrivercavejunct.csv'
output_file=processed_dir+'/RiverFlowData.txt'

In [204]:
#pd.read_csv(precip_data_fn).tail()

In [205]:
#weather_stations=['happycamp', 'orleans','salmonriver','cavejunction']
#precip_data_fns={'happycamp': '/happycampprecipdata.csv',\
#                 'orleans': '/orleansprecipdata.csv',\
#                 'salmonriver':'/salmonrivercavejunct.csv',\
#                 'cavejunction':'/salmonrivercavejunct.csv'}
#station_name = {'happycamp': 'USC00043761',\
#                 'orleans': 'USC00046508',\
#                 'salmonriver':'US1CASK0009',\
#                 'cavejunction':'US1ORJS0024'}

In [206]:
#precip_df = pd.read_csv(raw_dir+precip_data_fns[weather_stations[0]])[['DATE', 'PRCP', 'TMAX','TMIN','TOBS']].set_index('DATE')
#precip_df=precip_df.rename(columns={'PRCP':'PRCP_'+weather_stations[0], 'TMAX':'TMAX_'+weather_stations[0], \
#                          'TMIN':'TMIN_'+weather_stations[0], 'TOBS':'TOBS_'+weather_stations[0]})

In [207]:
#for station in weather_stations[1:]:
#    precip_station_df = pd.read_csv(precip_data_fn)[['STATION','DATE', 'PRCP']].set_index('DATE')
#    precip_station_df=precip_station_df[precip_station_df['STATION']==station_name[station]]
#    precip_station_df=precip_station_df.drop(columns=['STATION'])
#    precip_station_df=precip_station_df.rename(columns={'PRCP':'PRCP_' + station})
#    precip_df=precip_df.join(precip_station_df, how='outer')


In [208]:
#precip_df.head()

In [209]:
river_names=['IndianCreek', 'WillowCreek', 'CacheCreek', 'SuckerCreek', 'JockoRiver','SalmonFallsCreek']#,'SalmonRiver']
discharge_map={'IndianCreek': '296197_00060_00003','WillowCreek':'213773_00060_00003', 'CacheCreek': '11067_00060_00003', 'SuckerCreek':'114843_00060_00003', \
              'JockoRiver': '82127_00060_00003', 'SalmonFallsCreek': '45609_00060_00003'}#, 'SalmonRiver':'47824_00060'}
#gage_ht_map={'IndianCreek': '16575_00065','SalmonRiver':'47825_00065','SuckerCreek':'117620_00065'}
header_line_num={'IndianCreek': 28, 'WillowCreek': 27, 'CacheCreek': 28, 'SuckerCreek':28, 'JockoRiver': 29,'SalmonFallsCreek': 30}#, 'SalmonRiver':31,'SuckerCreek':29}

In [210]:

begin_date = '2000-01-01'
end_date='2022-12-07'

#output_df = pd.DataFrame({'DATE':pd.date_range(begin_date, end_date)}).set_index('DATE')
#output_df=output_df.join(precip_df.set_index(pd.DatetimeIndex(precip_df.index)), how='outer')

In [219]:
river = river_names[0]
flow_filename=raw_dir+'/'+ river+'FlowData.txt'
dataframe=pd.read_csv(flow_filename,header=header_line_num[river], skiprows=[header_line_num[river]+1],\
        delimiter='\t', low_memory=False)
dataframe.rename(mapper={discharge_map[river]:'discharge_'+river},inplace=True, axis="columns")
out_df = dataframe[['datetime','discharge_'+river]]
out_df['discharge_'+river+'_daily_change']=out_df['discharge_'+river].diff(periods=1)
#out_df.set_index(keys='datetime', inplace=True)

for river in river_names[1:]:
    flow_filename=raw_dir+'/'+ river+'FlowData.txt'
    dataframe=pd.read_csv(flow_filename,header=header_line_num[river], skiprows=[header_line_num[river]+1],\
            delimiter='\t', low_memory=False)
    dataframe.rename(mapper={discharge_map[river]:'discharge_'+river},inplace=True, axis="columns")
    flow_df = dataframe[['datetime','discharge_'+river]]
    flow_df['discharge_' + river].replace('Ice', 'nan', inplace=True)
    flow_df['discharge_' + river]= flow_df['discharge_' + river].astype(float)
    flow_df['discharge_'+river+'_daily_change']=flow_df['discharge_'+river].diff(periods=1)
    flow_df.set_index(keys='datetime', inplace=True)
    out_df = out_df.join(flow_df, on='datetime', how='outer')

/var/folders/1k/s33lj1kd7px7nbb93v5jzphc0000gn/T/ipykernel_71670/861855333.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df['discharge_'+river+'_daily_change']=out_df['discharge_'+river].diff(periods=1)
/var/folders/1k/s33lj1kd7px7nbb93v5jzphc0000gn/T/ipykernel_71670/861855333.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df['discharge_' + river].replace('Ice', 'nan', inplace=True)
/var/folders/1k/s33lj1kd7px7nbb93v5jzphc0000gn/T/ipykernel_71670/861855333.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [221]:
split_date=out_df['datetime'].str.split('-', expand=True)
out_df['year']=split_date[0]
out_df['month']=split_date[1]
out_df['day']=split_date[2]
out_df.to_csv(output_file)